In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import os
from fastai.text import *
from fastai.imports import *
from fastai.text import *
from fastai import *
import warnings
warnings.filterwarnings("ignore")

In [4]:
train = pd.read_csv('data/train_df.csv')
test = pd.read_csv('data/final dataset/test_df.csv')

In [5]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   id                                            14004 non-null  int64 
 1   ABSTRACT                                      14004 non-null  object
 2   Computer Science                              14004 non-null  int64 
 3   Mathematics                                   14004 non-null  int64 
 4   Physics                                       14004 non-null  int64 
 5   Statistics                                    14004 non-null  int64 
 6   Analysis of PDEs                              14004 non-null  int64 
 7   Applications                                  14004 non-null  int64 
 8   Artificial Intelligence                       14004 non-null  int64 
 9   Astrophysics of Galaxies                      14004 non-null  int64 
 10

In [6]:
import os 
cwd = os.getcwd()
cwd

'F:\\Ml files\\research topic modeling'

In [7]:
## dropped multilabeled columns
cols = ['ABSTRACT', 'Analysis of PDEs', 'Applications',
       'Artificial Intelligence', 'Astrophysics of Galaxies',
       'Computation and Language', 'Computer Vision and Pattern Recognition',
       'Cosmology and Nongalactic Astrophysics',
       'Data Structures and Algorithms', 'Differential Geometry',
       'Earth and Planetary Astrophysics', 'Fluid Dynamics',
       'Information Theory', 'Instrumentation and Methods for Astrophysics',
       'Machine Learning', 'Materials Science', 'Methodology', 'Number Theory',
       'Optimization and Control', 'Representation Theory', 'Robotics',
       'Social and Information Networks', 'Statistics Theory',
       'Strongly Correlated Electrons', 'Superconductivity',
       'Systems and Control']
train = train.drop(columns = cols)
train.head()


,id,Computer Science,Mathematics,Physics,Statistics,cleaned_text
0,1824,0,0,1,0,evergrowing datasets inside observational astr...
1,3094,1,0,0,0,propose framework consider optimal tmatchings ...
2,8463,0,0,1,0,nanostructures open shell transition metal mol...
3,2082,0,0,1,0,star selfgravitating fluid inside pressure buo...
4,8687,1,0,0,0,deep neural perception control network likely ...


In [8]:
print('Shape of Training data {}'.format(train.shape))
print('Shape of Testing data {}'.format(test.shape))


Shape of Training data (14004, 6)
Shape of Testing data (6002, 7)


In [9]:
train.loc[(train['Computer Science']==1)
               , 'label'] = 'Computer Science'

train.loc[(train['Mathematics']==1)
               , 'label'] = 'Mathematics'


train.loc[(train['Physics']==1)
               , 'label'] = 'Physics'
               
train.loc[(train['Statistics']==1)
               , 'label'] = 'Statistics'  

train.head()

,id,Computer Science,Mathematics,Physics,Statistics,cleaned_text,label
0,1824,0,0,1,0,evergrowing datasets inside observational astr...,Physics
1,3094,1,0,0,0,propose framework consider optimal tmatchings ...,Computer Science
2,8463,0,0,1,0,nanostructures open shell transition metal mol...,Physics
3,2082,0,0,1,0,star selfgravitating fluid inside pressure buo...,Physics
4,8687,1,0,0,0,deep neural perception control network likely ...,Computer Science


In [10]:
train.label = train.label.map({'Computer Science': 0, 'Mathematics' : 1, 'Physics': 2, 'Statistics' : 3, })
print(train.info())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                14004 non-null  int64 
 1   Computer Science  14004 non-null  int64 
 2   Mathematics       14004 non-null  int64 
 3   Physics           14004 non-null  int64 
 4   Statistics        14004 non-null  int64 
 5   cleaned_text      14004 non-null  object
 6   label             14004 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 766.0+ KB
None


,id,Computer Science,Mathematics,Physics,Statistics,cleaned_text,label
0,1824,0,0,1,0,evergrowing datasets inside observational astr...,2
1,3094,1,0,0,0,propose framework consider optimal tmatchings ...,0
2,8463,0,0,1,0,nanostructures open shell transition metal mol...,2
3,2082,0,0,1,0,star selfgravitating fluid inside pressure buo...,2
4,8687,1,0,0,0,deep neural perception control network likely ...,0


In [11]:
train = train.drop(['Computer Science',	'Mathematics',	'Physics',	'Statistics', 'id'], axis = 1)
train.head()

,cleaned_text,label
0,evergrowing datasets inside observational astr...,2
1,propose framework consider optimal tmatchings ...,0
2,nanostructures open shell transition metal mol...,2
3,star selfgravitating fluid inside pressure buo...,2
4,deep neural perception control network likely ...,0


In [12]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [ ]:
from sklearn.metrics import accuracy_score 
y_pred_totcb = []
from sklearn.model_selection import KFold, RepeatedKFold
fold = KFold(n_splits=2, shuffle=True, random_state=0)
i=1

for train_index, test_index in fold.split(train):
    
    train_df = train.iloc[train_index]
    valid_df = train.iloc[test_index]

    random_seed(1)
    
    data_lm = TextLMDataBunch.from_df(Path('/data'), train_df, valid_df, test, text_cols=[0], bs=32)
    data_clas = TextClasDataBunch.from_df(Path('/data'), train_df, valid_df, test, text_cols=[0], label_cols=1, bs=32)
    
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.4, model_dir='models/')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.unfreeze()
    learn.fit_one_cycle(9, 1e-3, moms=(0.8,0.7))
    learn.save_encoder('model_enc')
    
    learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.4, model_dir='models/')
    learn.load_encoder('model_enc')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.freeze_to(-2)
    learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
    learn.freeze_to(-3)
    learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
    learn.unfreeze()
    learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))
   
    log_preds, test_labels = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
    preds = np.argmax(log_preds, 1)
    y_pred_totcb.append(preds)
    print(f'fold {i} completed')
    i = i+1


In [ ]:
y_pred_totcb[0]

In [ ]:
df = pd.DataFrame()
for i in range(3):
    col_name = 'SECTION_' + str(i)
    df[col_name] =  y_pred_totcb[i]

In [ ]:
sub = pd.DataFrame()
sub['SECTION'] = df.mode(axis=1)[0]
sub.tail()

In [ ]:
sub.shape